# Logistic Regression

This is the code used for the Logistic Regression.
We used this code to analyze both the users with 100% agreement and 75% of agreement.
To analyze them it's enought to import the right dataframe.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
listed = drive.ListFile({'q': "title contains '.pkl' and 'root' in parents"}).GetList()
for file in listed:
    print('title {}, id {}'.format(file['title'], file['id']))



In [ ]:
#Needed to import left-right dataframe
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import pickle
from googleapiclient.http import MediaIoBaseDownload

file_id = '1fo-gIx5b8J9eiZcMFik9zK7OTCgcIJnT'

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

downloaded.seek(0)
f=pickle.load(downloaded)

In [ ]:
data.head(15)

In [ ]:
import pandas as pd
import numpy as np
df=f

In [ ]:
#m5s=0, sx=1, dx=2
#I change the 
for i in np.arange(0,3300) :
    if data.loc[i,'__label__']=='__label__Unknown' : data.loc[i,'__label__']=0
    elif data.loc[i,'__label__']=='__label__Left' : data.loc[i,'__label__']=1
    elif data.loc[i,'__label__']=='__label__Right' : data.loc[i,'__label__']=2
data.head(10)


In [ ]:
df=f
df=df[df.loc[:,'__label__']!='__label__Unknown']
df.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

training_data, testing_data = train_test_split(df,random_state = 2000)
Y_train=training_data['__label__']
Y_test=testing_data['__label__']
X_train=training_data['tweets']
X_test=testing_data['tweets']
field='tweets'
field

In [ ]:
print(X_test)

In [ ]:
def extract_features(df,field,training_data,testing_data,type="binary"):
  if "binary" in type:
        # BINARY FEATURE REPRESENTATION
        cv= CountVectorizer(binary=True, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set, test_feature_set, cv
   
  elif "counts" in type:
        # COUNT BASED FEATURE REPRESENTATION
        cv= CountVectorizer(binary=False, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
    
  else:    
        
        # TF-IDF BASED FEATURE REPRESENTATION
        tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)
        tfidf_vectorizer.fit_transform(training_data[field].values)
        
        train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
        test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,tfidf_vectorizer
  
    
   

In [ ]:
X_train,X_test,feature_transformer = extract_features(df, field,training_data,testing_data, type="counts")
    

In [ ]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

In [ ]:
model=scikit_log_reg.fit(X_train,Y_train)

In [ ]:
model.score(X_test,Y_test)